In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objects as go

import sklearn.metrics
import scipy.integrate as integrate
import scipy.optimize as sco

import fluids as fds
import ht 

import general as gen
import model as ty
import proc as pr
import heat_transfer as bht
# importing sys
import sys
sys.path.insert(0,r'C:\Users\valen\Dropbox (DualSun)\PC\Documents\GitHub\RD-systems-and-test-benches\thermal')
import SK_standard as sk

In [2]:
input_name = r'\Inputs_V4.41-NI-glass-glass.xlsx'
hyp_name = r'\Model_hypotheses.xlsx'
data_path = r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Steady-state conditions inputs'
data_name = r'\221223_TUV_test_conditions_V4.41GGA_MPP.xlsx'

par = pr.import_input(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Inputs',input_name)
hyp = pr.create_dict_from_excel(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Hypotheses',hyp_name,'Main')
condi = pr.create_inputs_from_excel(data_path+data_name,par,hyp)

{'exchanger': {'name': nan, 'eta_nom': 0.201, 'Eff_T': -0.0034000000000000002, 'T_ref': 298.15, 'Eff_G': 0, 'G_ref': 1000, 'X_corr': 1, 'tau_g': 0.8, 'alpha_g': 0.05, 'eps_PV': 0.9, 'eps_g': 0.9, 'eps_bs': 1, 'w_pan': 1.134, 'L_pan': 1.708, 'orientation': 'portrait', 'lambd_upper_glass': 0.002, 'lambd_upper_EVA': 0.000515, 'lambd_si': 0.00015, 'lambd_lower_EVA': 0.000515, 'lambd_PVDF': 0.0, 'lambd_PET': 0.0, 'lambd_adh': 0.0, 'lambd_lower_glass': 0.002, 'k_glass': 0.8, 'k_EVA': 0.24, 'k_si': 60, 'k_PVDF': 0.2, 'k_PET': 0.3, 'k_adh': 0.2, 'L_abs': 1.342, 'w_abs': 0.976, 'geometry': 'harp', 'N_harp': 300, 'N_meander': 1, 'L_tube': 1.342, 'tube_geometry': 'rectangular', 'H_tube': 0.002, 'w_tube': 0.00281, 'W': 0.00335, 'l_c': 0.003, 'p_ext_tube': 0.00335, 'p_ext_tube_rad': 0.00335, 'lambd_air': 0.0, 'lambd_conductive_plate': 0.0, 'lambd_abs': 0.00035, 'lambd_riser_plate': 0.0, 'lambd_riser_back': 0.00035, 'lambd_ins': 0.0, 'eps_hx_top': 0, 'eps_hx_back': 0.1, 'eps_ins': 0.2, 'k_air': 0.02

In [3]:
df_res,X,list_res = ty.simu_condi(par,hyp,condi)

top h simple DT<0 theta>=2 Ra_L 212624778377.56647
top h simple DT<0 theta>=2 Ra_L 249771540083.503
top h simple DT<0 theta>=2 Ra_L 153514442579.70395
top h simple DT<0 theta>=2 Ra_L 147750097450.28384


In [4]:
[tab,X] = sk.find_SK(condi,par['AG'],columns_list=[1,1,0,1,0,0,1,0,0])
X_df = pd.DataFrame(X[0],index = ['eta0,hem','a1','a2','a3','a4','a5','a6','a7','a8'],columns=['TÜV'])

[tab_m,X_m] = sk.find_SK(df_res,par['AG'],columns_list=[1,1,0,1,0,0,1,0,0])
X_df['Linear fit model 1D'] = X_m[0]

pr.disp_html(X_df)

# X_df.to_csv('V4.41_glass_backsheet_MPP.csv',sep=";")

,TÜV,Linear fit model 1D
"eta0,hem",0.486248,0.477656
a1,21.363516,22.373826
a2,0.000000,0.000000
a3,1.070499,1.226628
a4,0.000000,0.000000
a5,0.000000,0.000000
a6,0.012652,0.004593
a7,0.000000,0.000000
a8,0.000000,0.000000


In [7]:
22.37-2*1.3

19.77

In [ ]:
par["manifold"]["L_f0"]

In [ ]:
pr.disp_html(df_res)

In [ ]:
condi["dTm/dt"] = 0.

u_list = condi["u"].unique()
color_list_1 = ['lightgreen','skyblue','darksalmon']
color_list_2 = ['green','darkblue','tomato']

fig = go.Figure()

for i in range(len(u_list)):
    # Add traces

    fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=df_res.loc[df_res["u"]==u_list[i]]['Qdot / AG'],
                        mode = 'markers',marker_color = color_list_1[i],
                        name='Model 1D - u = '+str(u_list[i])+' m/s',
                        ))

    
    fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=sk.comp_power_rG_coeff_df(X_df['Linear fit model 1D'],df_res.loc[df_res["u"]==u_list[i]]),
                        mode = 'lines',
                        line=dict(color=color_list_1[i], width=1,
                            dash='dashdot'),
                        name='Linear fit model 1D - u = '+str(u_list[i])+' m/s',
                        ))
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['Tm - T_a'], y=condi.loc[condi["u"]==u_list[i]]['Qdot / AG'],
                        mode = 'markers',marker_color = color_list_2[i],
                        name='TUV - u = '+str(u_list[i])+' m/s')
                        )
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['Tm - T_a'], y=sk.comp_power_rG_coeff_df(X_df['TÜV'],condi.loc[condi["u"]==u_list[i]]),
                        mode = 'lines',
                        line=dict(color=color_list_2[i],width=1
                            ),
                        name='Linear fit TUV - u = '+str(u_list[i])+' m/s')
                        )
# Set x-axis title
fig.update_xaxes(title_text="T_m - T_amb")
fig.update_yaxes(title_text="Power related to gross (W/m2 coll.)")

fig.update_layout(
    title={
        'text': "V4.41 glass-backsheet dark no wind",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_layout(
    autosize=False,
    width=1200,
    height=700,
    margin=dict(
        l=0,
        r=0,
        b=50,
        t=50,
        pad=1
    ),
)

fig.show()

In [ ]:
fig.write_html('V4.41_glass_backsheet_MPP.html')

In [ ]:
pr.disp_html(df_res)

In [ ]:
[tab,X] = sk.find_SK_SST(condi,par['A_G'],columns_list=[0,1,0,1,0,0,0,0,0])
[tab_m,X_m] = sk.find_SK_SST(df_res,par['A_G'],columns_list=[0,1,0,1,0,0,0,0,0])

In [ ]:
pr.display_a_i(X)
pr.display_a_i(X_m)

In [ ]:
condi

In [ ]:
pr.disp_html(df_res)

In [ ]:
u_list = [0.7,2.1,3.9]

color_list = ['blue','red','green','orange','cyan','purple']

def lin(x,a,b):
    return a*x+b

popt_mod_list = []
pcov_mod_list = []
popt_tuv_list = []
pcov_tuv_list = []


for i in range(len(u_list)):
    popt,pcov = sco.curve_fit(lin,-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'],df_res.loc[df_res["u"]==u_list[i]]['Q_dot / A_G'])
    popt_mod_list.append(popt)
    pcov_mod_list.append(pcov)

    popt,pcov = sco.curve_fit(lin,condi.loc[condi["u"]==u_list[i]]['T_m - T_a'],condi.loc[df_res["u"]==u_list[i]]['Q_dot / A_G'])
    popt_tuv_list.append(popt)
    pcov_tuv_list.append(pcov)

fig = go.Figure()

for i in range(len(u_list)):
    # Add traces

    fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=df_res.loc[df_res["u"]==u_list[i]]['Q_dot / A_G'],
                        mode = 'markers',
                        marker=dict(color=color_list[2*i]),
                        name='Model 1D - u = '+str(u_list[i])+' m/s')
                        )
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['T_m - T_a'], y=condi.loc[condi["u"]==u_list[i]]['Q_dot / A_G'],
                        mode = 'markers',
                        marker=dict(color=color_list[2*i+1]),
                        name='TUV - u = '+str(u_list[i])+' m/s')
                        )


    fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=lin(-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'],*popt_mod_list[i]),
                        mode = 'lines',
                        line=dict(color=color_list[2*i], width=1,
                              dash='dashdot'),
                        name='Linear fit model 1D - u = '+str(u_list[i])+' m/s')
                        )
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['T_m - T_a'], y=lin(condi.loc[condi["u"]==u_list[i]]['T_m - T_a'],*popt_tuv_list[i]),
                        mode = 'lines',
                        line=dict(color=color_list[2*i+1], width=1,
                              dash='dashdot'),
                        name='Linear fit TUV - u = '+str(u_list[i])+' m/s')
                        )
# Set x-axis title
fig.update_xaxes(title_text="T_m - T_amb")
fig.update_yaxes(title_text="Power related to gross (W/m2 coll.)")

fig.update_layout(
    title={
        'text': "V4.41 glass-backsheet dark no wind",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_layout(
    autosize=False,
    width=1200,
    height=700,
    margin=dict(
        l=0,
        r=0,
        b=50,
        t=50,
        pad=1
    ),
)

fig.show()

In [ ]:
mse = sklearn.metrics.mean_squared_error(df_res['Q_dot / A_G'],condi['Q_dot / A_G'])
print(math.sqrt(mse))